In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import time
import csv
import pandas as pd
np.set_printoptions(precision=5, suppress=True, linewidth=100)
plt.rcParams['figure.dpi'] = 150
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import tebd
from tenpy.networks.mps import MPS
from tenpy.models.model import CouplingMPOModel
from tenpy.models.model import NearestNeighborModel
from tenpy.networks import site
import matplotlib.pyplot as plt
tenpy.tools.misc.setup_logging(to_stdout="INFO")

from tenpy.models.lattice import Square
from numpy.linalg import svd
from tenpy.algorithms import dmrg
from tenpy.linalg.truncation import svd_theta
from tenpy.linalg.sparse import FlatLinearOperator
from tenpy.tools.math import speigs
import copy

### Average effective Hamiltonian in the doubled Hilbert space of a Z2 symmetric Brownian circuit

In [5]:
class Ising_2copy(CouplingMPOModel):
    """Average time evolution in doubled Hilbert space of random TFI"""

    default_lattice = "Chain"
    force_default_lattice = True

    def init_sites(self, model_params):
        X= np.array([[0,1],[1,0]])
        spin1 = site.SpinHalfSite(conserve="parity")
        spin2 = site.SpinHalfSite(conserve="parity")
        #spin1.add_op('Sx',1/2*X)
        #spin2.add_op('Sx',1/2*X)
        sites = [spin1, spin2]
        
        site.set_common_charges(sites, new_charges='independent')
        return [spin1, spin2], ['sf', 'sb']  # return species names as well!
        # this causes `init_lattice` to initialize a `MultiSpeciesLattice`
        
    def init_terms(self, model_params):
        J = model_params.get('J', 1.)
        U = model_params.get('U', 1.)
        s = model_params.get('s', 0.5)
        p = model_params.get('p', 0.1)
        # onsite f-b interaction
        self.add_coupling(-2*U, 0, 'Sigmaz', 1, 'Sigmaz', [0])
        # magnetic field from postselection
        self.add_onsite(-p*s/4,0,'Sigmaz')
        self.add_onsite(-p*s/4,1,'Sigmaz')
        # f-b hopping term
        self.add_multi_coupling(-2*J,[('Sigmax',[0],0),('Sigmax',[1],0),('Sigmax',[0],1),('Sigmax',[1],1)])


### Run a DMRG simulation to find the groundstate

In [25]:
product_state = [('up','up')]
print('product_state = ')
print(product_state)
print()
initial_time=time.time()
Ls=[16]
Np=20
cor=np.zeros((len(Ls),Np))
ps =np.linspace(0.01,16,Np)
states=[]
for i,L in enumerate(Ls):
    print("L=",L)
    print()
    for j,prob in enumerate(ps):
        print("p=",prob)
        M=Ising_2copy({'L':L,'p':prob,'s':0.5})
        psi0 = MPS.from_lat_product_state(M.lat, product_state)
        eng = tenpy.algorithms.dmrg.SingleSiteDMRGEngine(psi0, M, {'mixer_params': {"amplitude": 1e-5},
                                                                        'trunc_params': {'chi_max': 100, 'svd_min':1e-7}, 'max_sweeps':30,'max_E_err':1.0e-12})
        E, psi = eng.run()
        states.append(psi.copy())
        cor[i,j]=float(psi.expectation_value_term([('Sigmax',int((L/4-1)*2)),('Sigmax',int((L/4-1)*2+1)),('Sigmax',int(L-L/4-1)*2),('Sigmax',int(L-L/4-1)*2+1)]))
    print()
    print("---------------------------------------------------")
    print(time.time()-initial_time)

product_state = 
[('up', 'up')]

L= 16

p= 0.01
INFO    : Ising_2copy: reading 'L'=16
INFO    : Ising_2copy: reading 's'=0.5
INFO    : Ising_2copy: reading 'p'=np.float64(0.01)
INFO    : SingleSiteDMRGEngine: subconfig 'trunc_params'=Config(<2 options>, 'trunc_params')
INFO    : SingleSiteDMRGEngine: subconfig 'mixer_params'=Config(<1 options>, 'mixer_params')
INFO    : activate SubspaceExpansion with initial amplitude 1e-05
INFO    : SingleSiteDMRGEngine: reading 'max_sweeps'=30
INFO    : Running sweep with optimization
INFO    : trunc_params: reading 'chi_max'=100
INFO    : trunc_params: reading 'svd_min'=1e-07
INFO    : checkpoint after sweep 1
energy=-62.0000140625013358, max S=1.0394045397033917, age=32, norm_err=1.0e+00
Current memory usage 274.0MB, wall time: 0.7s
Delta E = nan, Delta S = 7.7860e-01 (per sweep)
max trunc_err = 9.5397e-47, max E_trunc = 5.4742e-03
chi: [2, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 2]
INFO    : Running 

### get the matrix product density operator from the groundstates

In [12]:
def MPS_drop_charge(psi, charge=None, chinfo=None, permute_p_leg=True):
    psi_c = psi.copy()
    psi_c.chinfo = chinfo = npc.ChargeInfo.drop(chinfo, charge=charge)
    if permute_p_leg is None and chinfo.qnumber == 0:
        permute_p_leg = True
    for i, B in enumerate(psi_c._B):
        psi_c._B[i] = B = B.drop_charge(charge=charge, chinfo=chinfo)
        psi_c.sites[i] = site = copy.copy(psi.sites[i])
        if permute_p_leg:
            if permute_p_leg is True:
                perm = tenpy.tools.misc.inverse_permutation(site.perm)
            else:
                perm = permute_p_leg
            psi_c._B[i] = B = B.permute(perm, 'p')
        else:
            perm = None
        site.change_charge(B.get_leg('p'), perm) # in place
    psi_c.test_sanity()
    return psi_c
    
def double_to_mpo(psi,L):
    Ws=[]
    psi=MPS_drop_charge(psi)
    for i in range(L):
        a=npc.tensordot(psi.get_B(2*i),psi.get_B(2*i+1),axes=(2,0))
        a.itranspose((0,3,1,2))
        a_array=a.to_ndarray().astype(np.float64)
        
        Ws.append(a_array)
    return Ws



In [26]:
mpo_arrays=[]
L=16
for state in states:
    state = MPS_drop_charge(state)
    W = double_to_mpo(state,L)
    mpo_arrays.append(W)

In [28]:
np.save('../data/groundstates_Z2_L'+str(L), np.array(mpo_arrays, dtype=object), allow_pickle=True)